In [1]:
import pandas as pd
from app import db
import numpy as np

In [2]:
df = pd.read_sql('access_log', db.engine, index_col='id')

In [3]:
df = df[['user_id', 'time', 'key', 'value']]

In [4]:
df = df.assign(count_by_user=df.assign(val=1).groupby('user_id').agg({'val': 'cumsum'})).reset_index()

In [5]:
df = df.set_index(['user_id', 'count_by_user']).sort_index()

In [6]:
enhanced_df = df.assign(
    prev_key=df['key'].shift(1).fillna(-1).astype(int),
    prev_value=df['value'].shift(1).fillna(-1).astype(int),
    two_keys_ago=df['key'].shift(2).fillna(-1).astype(int),
    two_values_ago=df['value'].shift(2).fillna(-1).astype(int),
    diff_key=df['key'].diff().fillna(-1).astype(int),
)
# Also set first of each user's actions to -1
enhanced_df.loc[pd.IndexSlice[:, [1]], ['prev_key', 'prev_value', 'diff_key']] = -1
enhanced_df.loc[pd.IndexSlice[:, [1, 2]], ['two_keys_ago', 'two_values_ago']] = -1

enhanced_df = enhanced_df.assign(
    ratio_key=enhanced_df['key'] / enhanced_df['prev_key'],
    time_seconds=(enhanced_df['time'] - enhanced_df['time'].min()).dt.seconds
)
enhanced_df =  enhanced_df.assign(
#     mult=((enhanced_df['ratio_key'] * enhanced_df['key']) % 1e6).astype(int),
#     summer=((enhanced_df['key'] + enhanced_df['diff_key']) % 1e6).astype(int),
    seconds_since_last=enhanced_df['time_seconds'] - enhanced_df['time_seconds'].shift(1).fillna(-1).astype(int),
)
enhanced_df.loc[(enhanced_df.ratio_key < 0), ['ratio_key', ]] = -1
enhanced_df.loc[pd.IndexSlice[:, [1]], 'seconds_since_last'] = -1
enhanced_df = enhanced_df.reset_index()


In [7]:
target = pd.concat([df['key'].shift(-1).fillna(-1).astype(int) - df['key'], df['key'].shift(-1).fillna(-1).astype(int).rename('next_key'), df['time']], axis=1)
target.loc[pd.IndexSlice[:, [10]], ['key']] = -1
target = target.reset_index(drop=True)
target = target.rename({'key': 'delta_key'}, axis=1)

In [8]:
cutoff = enhanced_df.time.quantile(.7)
mask = (~(enhanced_df.drop('time', axis=1) < 0).any(axis=1)) & (~(target.drop('time', axis=1) < 0).any(axis=1))
train_df = enhanced_df.loc[(enhanced_df.time <= cutoff) & (mask)].drop(['time', 'count_by_user'], axis=1)
train_y = target.loc[(target.time <= cutoff) & (mask)]['delta_key']
test_df = enhanced_df.loc[(enhanced_df.time > cutoff) & (mask)].drop(['time', 'count_by_user'], axis=1)
test_y = target.loc[(target.time > cutoff) & (mask)]['delta_key']

In [28]:
import lightgbm as lgb
ds = lgb.Dataset(train_df, label=train_y, free_raw_data=False)
valid = lgb.Dataset(test_df, label=test_y, free_raw_data=False)

In [29]:
params = {
    'objective': 'rmse',
    'num_boost_round': 300,
    'early_stopping_rounds': 30,
    'learning_rate': 0.3,
    'num_leaves': 12,
    'min_data_in_leaf': 50,
    'lambda_l1': 0.5,
    'lambda_l2': 0.5,
    'bagging_fraction': 0.2,
    'metric': ['mape', 'rmse']
}
model = lgb.train(params, ds, valid_sets=valid, )

[1]	valid_0's mape: 52.0853	valid_0's rmse: 17519.9
Training until validation scores don't improve for 30 rounds
[2]	valid_0's mape: 47.7295	valid_0's rmse: 14620.8
[3]	valid_0's mape: 42.5975	valid_0's rmse: 13292.1
[4]	valid_0's mape: 38.1914	valid_0's rmse: 12232.5
[5]	valid_0's mape: 31.5539	valid_0's rmse: 11593.6
[6]	valid_0's mape: 32.8878	valid_0's rmse: 10612.9
[7]	valid_0's mape: 32.6877	valid_0's rmse: 10260.6
[8]	valid_0's mape: 33.2279	valid_0's rmse: 9935.43
[9]	valid_0's mape: 34.5638	valid_0's rmse: 8748.62
[10]	valid_0's mape: 35.2681	valid_0's rmse: 8653.5
[11]	valid_0's mape: 33.3648	valid_0's rmse: 8522.4
[12]	valid_0's mape: 31.7705	valid_0's rmse: 8397.86
[13]	valid_0's mape: 30.408	valid_0's rmse: 8300.37
[14]	valid_0's mape: 31.163	valid_0's rmse: 8202.33
[15]	valid_0's mape: 30.6484	valid_0's rmse: 7994.58
[16]	valid_0's mape: 33.2055	valid_0's rmse: 6877.71
[17]	valid_0's mape: 34.1724	valid_0's rmse: 6726.79
[18]	valid_0's mape: 32.8278	valid_0's rmse: 6625.7

In [11]:
top_features=pd.Series(model.feature_importance(), index=train_df.columns).nlargest(5)

In [12]:
top_features

diff_key        1210
ratio_key        896
key              415
prev_key         325
two_keys_ago     235
dtype: int32

In [15]:
from sklearn.cluster import Birch
from sklearn.preprocessing import StandardScaler

In [17]:
scaler = StandardScaler()
cheaty = pd.concat([enhanced_df, target], axis=1)
clustery = scaler.fit_transform(cheaty[np.r_[top_features.index.values, ['next_key', 'delta_key']]])

In [18]:
%%time
# clustery = enhanced_df[['ratio_key', 'diff_key']]
stride = clustery.shape[0] / 100
model = Birch(n_clusters=None)
for i in range(100):
    print('.', end='')
    batch = clustery[int(i * stride):int((i + 1) * stride)]    
    model.partial_fit(batch)

....................................................................................................CPU times: user 1min 14s, sys: 8.44 s, total: 1min 23s
Wall time: 31.1 s


In [19]:
model.set_params(n_clusters=20) # Should capture a few different behaviors
model.partial_fit()

Birch(n_clusters=20)

In [20]:
cluster_bomb = enhanced_df.assign(
    cluster=model.predict(clustery)
)

In [21]:
cluster_bomb.cluster.value_counts()

5     475079
19    148422
10     82344
17     43658
8      40426
0      39914
4      34267
11     28716
9      28246
6      16553
3      15916
13     13277
18     10311
16      8866
2       8006
12      5876
15        84
14        17
7         12
1         10
Name: cluster, dtype: int64

In [22]:
cluster_bomb[cluster_bomb.cluster == 2]

,user_id,count_by_user,id,time,key,value,prev_key,prev_value,two_keys_ago,two_values_ago,diff_key,ratio_key,time_seconds,seconds_since_last,cluster
35,3,6,384929,2020-05-12 22:47:38.821288,200035,145175,600105,417774,500088,346945,-400070,0.333333,4138,60,2
82,8,3,644040,2020-05-12 23:30:43.821288,322019,185791,966057,845347,805048,2436,-644038,0.333333,6723,60,2
102,10,3,188335,2020-05-12 22:14:37.821288,279411,436377,838233,940583,698528,902525,-558822,0.333333,2157,60,2
143,14,4,743252,2020-05-12 23:47:11.821288,321835,144421,965505,92748,804588,548009,-643670,0.333333,7711,60,2
192,19,3,488846,2020-05-12 23:05:00.821288,315055,599830,945165,488455,787638,86070,-630110,0.333333,5180,60,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
998885,99888,6,28833,2020-05-12 21:48:06.821288,199503,982259,598509,77686,498758,3055,-399006,0.333333,566,60,2
999154,99915,5,240170,2020-05-12 22:23:12.821288,260227,739492,780681,105835,650568,297658,-520454,0.333333,2672,60,2
999303,99930,4,427015,2020-05-12 22:54:43.821288,273091,458864,819273,392253,682728,720719,-546182,0.333333,4563,60,2
999312,99931,3,94090,2020-05-12 21:58:58.821288,258999,62454,776997,270659,647498,853949,-517998,0.333333,1218,60,2


In [24]:
cluster_bomb.ratio_key.value_counts()

-1.000000    100000
 0.333333     61684
 1.200000     21289
 1.006104        20
 1.001471        19
              ...  
 1.000057         1
 1.000036         1
 1.000081         1
 1.199998         1
 1.000094         1
Name: ratio_key, Length: 335760, dtype: int64

In [25]:
from gplearn import genetic
from gplearn import functions, fitness

# modulo = functions.make_function(lambda x, y: np.where(y == 0, 0, x % y), name='modulo', arity=2)
divisible_by = functions.make_function(lambda x, y: np.where(y == 0, 0, 1*(x % y == 0)), name='divisible_by', arity=2)
# compare = functions.make_function(lambda x, y: 1*(x < y), name='compare', arity=2)
# equal = functions.make_function(lambda x, y: 1*(x == y), name='equal', arity=2)
where_true = functions.make_function(lambda x, y, z: np.where(x == 1, y, z), name='where_true', arity=3)
around_town = functions.make_function(lambda x, y: np.where(x < 0, x, y), name='around', arity=2)
where_less = functions.make_function(lambda w, x, y, z: np.where(w < x, y, z), name='where_less', arity=4)
where_equal = functions.make_function(lambda w, x, y, z: np.where(w == x, y, z), name='where_equal', arity=4)
greater_of = functions.make_function(lambda x, y: np.where(x < y, y, x), name='greater_of', arity=2)
roundy = functions.make_function(lambda x: x.round(), name='round', arity=1)
sub_cap = functions.make_function(lambda x: x - 1e6, name='sub_cap', arity=1)

In [26]:
def _correct_pct(y, y_pred, w):
    """Calculate the mean absolute error."""
    return np.average(((np.ceil(y_pred) == y) * 10 + np.clip(30 - np.abs(y_pred - y), 0, 30))/40, weights=w)

correct_pct = fitness.make_fitness(_correct_pct, greater_is_better=True)

# GPLearn for now

We probably want to add coevolution via something like DEAP, or just use featuretools and don't worry about validating.

In [27]:
for cluster, data in cluster_bomb.groupby('cluster'):
    if cluster in [0,1,2,3,4]:
        continue
    print(cluster)
    cutoff = data.time.quantile(0.7)
    m = data.time <= cutoff
    n = data.time > cutoff
    data = data.drop(['time', 'time_seconds', 'cluster', 'id', 'count_by_user'], axis=1)
    train_df = data.reindex(m.index)
    test_df = data.reindex(n.index)
    train_y = target.reindex(train_df.index).drop('time', axis=1)['delta_key']
    test_y = target.reindex(test_df.index).drop('time', axis=1)['delta_key']
    
    ds = lgb.Dataset(train_df, label=train_y, free_raw_data=False)
    valid = lgb.Dataset(test_df, label=test_y, free_raw_data=False)
    
    params = {
        'objective': 'rmse',
        'num_boost_round': 100,
        'early_stopping_rounds': 30,
        'learning_rate': 0.01,
        'num_leaves': 4,
        'min_data_in_leaf': 50,
        'lambda_l1': 0.5,
        'lambda_l2': 0.5,
        'bagging_fraction': 0.9,
        'metric': ['mape', 'rmse']
    }
    model = lgb.train(params, ds, valid_sets=valid, )
#     est_gp.fit(train_df, train_y)
    break

5


/Users/johnurbanik/prescient/env/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/Users/johnurbanik/prescient/env/lib/python3.8/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's mape: 74.8136	valid_0's rmse: 21869.3
Training until validation scores don't improve for 30 rounds
[2]	valid_0's mape: 73.9595	valid_0's rmse: 21823
[3]	valid_0's mape: 73.1319	valid_0's rmse: 21777.4
[4]	valid_0's mape: 72.3321	valid_0's rmse: 21732.5
[5]	valid_0's mape: 71.5619	valid_0's rmse: 21688.3
[6]	valid_0's mape: 70.783	valid_0's rmse: 21644.8
[7]	valid_0's mape: 70.0536	valid_0's rmse: 21602
[8]	valid_0's mape: 69.2937	valid_0's rmse: 21559.9
[9]	valid_0's mape: 68.5826	valid_0's rmse: 21518.5
[10]	valid_0's mape: 67.8414	valid_0's rmse: 21477.7
[11]	valid_0's mape: 67.1482	valid_0's rmse: 21437.5
[12]	valid_0's mape: 66.4252	valid_0's rmse: 21398.1
[13]	valid_0's mape: 65.921	valid_0's rmse: 21362.9
[14]	valid_0's mape: 65.4218	valid_0's rmse: 21328.3
[15]	valid_0's mape: 64.7604	valid_0's rmse: 21290.1
[16]	valid_0's mape: 64.2871	valid_0's rmse: 21256.6
[17]	valid_0's mape: 63.8265	valid_0's rmse: 21223.8
[18]	valid_0's mape: 63.3705	valid_0's rmse: 21191.5


In [623]:
for cluster, data in cluster_bomb.groupby('cluster'):
    if cluster in [0,1,2,3,4,5,6,7,8]:
        continue
    print(cluster)
    train_df = data.drop(['time', 'time_seconds', 'cluster', 'id', 'user_id', 'count_by_user'], axis=1)
#     sample_fraction = 1
#     if train_df.shape[0] > 10000:
#         sample_fraction = 10000/train_df.shape[0]
        
    gene_df = train_df.sample(n=np.minimum(20000, train_df.shape[0]))

    est_gp = genetic.SymbolicRegressor(
        population_size=500,
        tournament_size=20,
        generations=20, stopping_criteria=1.1,
        p_crossover=0.5, p_subtree_mutation=0.1,
        p_hoist_mutation=0.1, p_point_mutation=0.2,
        p_point_replace=0.05,
        const_range=(1/3, 1/3),
        max_samples=1, verbose=1,
        init_depth=(2,6), n_jobs=4,
        feature_names=train_df.columns,
        function_set=['add', 'sub', 'mul', 'div', 'neg', divisible_by, where_less, where_true, where_equal, greater_of, roundy],
        metric=correct_pct, low_memory=True,
        parsimony_coefficient='auto', random_state=np.random.randint(0,1e6)
    )
    gene_y = target.loc[gene_df.index].drop('time', axis=1)['delta_key']
    est_gp.fit(gene_df, gene_y)
    break

9
    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    59.99         0.270915       11                1              N/A     10.01s
   1    15.22         0.641115       16                1              N/A      6.41s
   2     5.76         0.785311        4                1              N/A     14.28s
   3    16.32           0.7912       12                1              N/A      8.99s
   4     7.92         0.797111       14                1              N/A     12.68s
   5     2.56         0.879274        6                1              N/A     11.16s
   6     6.53         0.753234        4                1              N/A      5.51s
   7    11.32         0.806487        8                1              N/A      7.85s
   8     5.57         0.786496        6                1              N/A

In [624]:
pd.concat([cluster_bomb[cluster_bomb.cluster == 9], target[cluster_bomb.cluster == 9]], axis=1)

,user_id,count_by_user,id,time,key,value,prev_key,prev_value,two_keys_ago,two_values_ago,diff_key,ratio_key,time_seconds,seconds_since_last,cluster,delta_key,next_key,time
19,1,10,345263,2020-05-12 22:40:59.821288,4657,290495,4627,599322,4597,664455,30,1.006484,3739,60,9,-1,454084,2020-05-12 22:40:59.821288
39,3,10,408576,2020-05-12 22:51:38.821288,200155,150453,200125,566060,200095,374577,30,1.000150,4378,60,9,-1,996608,2020-05-12 22:51:38.821288
99,9,10,637950,2020-05-12 23:29:42.821288,35083,968146,35053,382954,35023,303371,30,1.000856,6662,60,9,-1,698528,2020-05-12 23:29:42.821288
109,10,10,230671,2020-05-12 22:21:37.821288,93317,332909,93287,453837,93257,866731,30,1.000322,2577,60,9,-1,360811,2020-05-12 22:21:37.821288
119,11,10,630909,2020-05-12 23:28:32.821288,361081,537842,361051,932584,361021,691946,30,1.000083,6592,60,9,-1,863711,2020-05-12 23:28:32.821288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999829,99982,10,883571,2020-05-13 00:10:30.821288,12239,19840,12209,991327,12179,734219,30,1.002457,9110,60,9,-1,666696,2020-05-13 00:10:30.821288
999849,99984,10,991637,2020-05-13 00:29:30.821288,89555,836286,89525,157914,89495,811943,30,1.000335,10250,60,9,-1,816227,2020-05-13 00:29:30.821288
999899,99989,10,299953,2020-05-12 22:33:16.821288,6715,8458,6685,657343,6655,303040,30,1.004488,3276,60,9,-1,288833,2020-05-12 22:33:16.821288
999909,99990,10,411808,2020-05-12 22:52:11.821288,289103,70669,289073,712018,289043,692128,30,1.000104,4411,60,9,-1,953726,2020-05-12 22:52:11.821288


In [626]:
print(est_gp._program)

neg(divisible_by(key, key))


In [618]:
((np.ceil(est_gp.predict(train_df)) -  target.loc[train_df.index].drop('time', axis=1)['delta_key']) == 0).sum()

7637

In [619]:
print((target.loc[train_df.index].drop('time', axis=1)['delta_key'] /  train_df['key']).value_counts())

-0.666667    7637
-0.860921       3
-0.806597       3
-0.958856       3
-0.969293       3
             ... 
-0.973305       1
-0.855780       1
-0.900025       1
-0.912417       1
-0.803466       1
Length: 7880, dtype: int64


In [508]:
pd.Series(est_gp.predict(train_df).astype(int)).value_counts()

 20455     19
 8303      18
 275       18
 4975      18
 19789     17
           ..
 508060     1
-116899     1
 399527     1
-100523     1
 49176      1
Length: 196054, dtype: int64

In [ ]:
est_gp = genetic.SymbolicRegressor(
    population_size=1000,
    generations=20, stopping_criteria=0.01,
    p_crossover=0.7, p_subtree_mutation=0.1,
    const_range=(-10, 10),
    p_hoist_mutation=0.05, p_point_mutation=0.1,
    max_samples=0.1, verbose=1,
    init_depth=(1,4), n_jobs=4,
    feature_names=train_df.columns,
    function_set=['add', 'sub', 'mul', 'div', 'neg', divisible_by, where_less, where_equal],
    metric='rmse',
    parsimony_coefficient=0.01, random_state=0
)

In [ ]:
est_gp.fit(train_df, train_y)

In [ ]:
print(est_gp._program)

In [ ]:
prediction = pd.Series(model.predict(test_df, ntree_limit = 0), index=test_y.index, name='key')

In [ ]:
pd.Series(model.feature_importance(), index=train_df.columns).round(3)
# pd.Series(model.coef_, index=train_df.columns).round(3)


In [ ]:
(test_y.loc[(test_y > -1) & (test_df.mult > -1)]  - prediction.round().astype(int).loc[(test_y > -1) & (test_df.mult > -1)]).describe()

In [ ]:
(test_y == prediction.round().astype(int)).sum()/ test_y.shape[0]

In [ ]:
(test_y  - prediction.round().astype(int)).hist(bins=80, log=True)